<a href="https://colab.research.google.com/github/sweetmjkim/study_data_analytics/blob/main/docs/quests/CDAs/Q_CDA_categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# X(범주형) / Y(범주형) 케이스

## 문제 2번 : 요일(X)이 가장 인기 있는 과일 카테고리 판매(Y)에 영향을 미칩니까?
- 분석 방법: 카이제곱 검정 또는 기대 셀 빈도가 5 미만인 경우 피셔의 정확 검정을 사용하여 특정 요일에 과일 카테고리의 인기에 통계적으로 유의미한 차이가 있는지 평가합니다.
- 분석 내용 : 요일(X)이 과일 판매(Y)에 영향을 주는가?
 + 귀무가설 : 요일(X)이 과일 판매(Y)에 영향을 주지 않는다. (p-value > 0.05)
 + 대립가설 : 요일(X)이 과일 판매(Y)에 영향을 준다.(p-value <= 0.05)

In [15]:
# row data 확인
import pandas as pd
df_fs = pd.read_csv('/content/fruitstore_saleshistory.csv')
df_fs[:3]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05


In [16]:
# 결측치 확인
df_fs[['요일', '물품중분류']].isnull().sum()
df_fs.dropna(subset=['물품중분류'] ,inplace=True)
df_fs['물품중분류']

0            과일
1            버섯
2             알
3         두부/유부
4             빵
          ...  
705566      건어물
705567       해조
705568    두부/유부
705569       과자
705570       음료
Name: 물품중분류, Length: 705571, dtype: object

In [17]:
# 결측치 제거
df_fs[['요일', '물품중분류']].isnull().sum()

요일       0
물품중분류    0
dtype: int64

In [22]:
# 물품중분류에서 과일 카테고리만 확인
df_fs_category_fruit = df_fs[df_fs['물품중분류'] == '과일']
df_fs_category_fruit[:3]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
62,2018-01-02,화,1,1,445131904,정상회원,과실,과일,건대추,건대추(500g),1.0,수지구,풍덕천동,여,58.0,50대,22973,매장C,NaN,12:18
132,2018-01-02,화,1,1,744508416,정상회원,과실,과일,귤(조생),귤/유(1.5kg),1.0,기흥구,보정동,여,63.0,60대,11104,매장C,NaN,14:58


In [24]:
df_fs_category_fruit = df_fs[df_fs['물품중분류'] == '과일']
df_fs_category_pivot = pd.pivot_table(df_fs_category_fruit, index='요일', columns='물품소분류', aggfunc='size')
df_fs_category_pivot

물품소분류,건대추,곶감,귤(비가림),귤(조생),귤만감,냉동건조과일,단감,레몬,배,복숭아,블루베리,사과,산딸기,자두,참다래,청매실,홍시용감(대봉),황매실
요일,,,,,,,,,,,,,,,,,,
금,34.0,56.0,31.0,175.0,305.0,10.0,107.0,21.0,195.0,NaN,82.0,1197.0,72.0,NaN,206.0,11.0,5.0,45.0
목,38.0,67.0,55.0,162.0,264.0,5.0,82.0,36.0,179.0,NaN,85.0,884.0,101.0,NaN,289.0,31.0,6.0,37.0
수,58.0,87.0,28.0,201.0,233.0,1.0,115.0,23.0,216.0,2.0,72.0,861.0,61.0,NaN,188.0,21.0,8.0,38.0
월,40.0,84.0,37.0,188.0,317.0,4.0,141.0,22.0,227.0,NaN,57.0,1100.0,80.0,NaN,257.0,15.0,10.0,7.0
일,20.0,39.0,20.0,112.0,125.0,1.0,60.0,10.0,102.0,NaN,15.0,538.0,10.0,NaN,111.0,NaN,6.0,NaN
토,44.0,71.0,31.0,182.0,271.0,4.0,119.0,18.0,174.0,2.0,89.0,1041.0,73.0,9.0,203.0,4.0,4.0,8.0
화,54.0,63.0,25.0,214.0,257.0,8.0,139.0,21.0,173.0,NaN,67.0,913.0,68.0,NaN,192.0,12.0,8.0,NaN


In [28]:
# 물품소분류에 있는 결측치 0으로 처리
df_fs_category_pivot['물품소분류'] = df_fs[df_fs['물품소분류']].fillna(0)
# df_fs_category_pivot['소분류'] = df_fs_category_pivot['건대추	곶감','귤(비가림)','귤(조생)','귤만감','냉동건조과일','단감','레몬','배','복숭아','블루베리','사과','산딸기','자두','참다래','청매실','홍시용감(대봉)','황매실'].fillna(0)

KeyError: "None of [Index(['사과', '느타리버섯', '유정란', '두부', '식빵, 식사대용', '유정란', '딸기', '어린잎채소', '생강',\n       '쌈케일',\n       ...\n       '브로컬리', '기타해조류', '생숙면', '김', '케이크,롤,선물', '멸치류', '미역/다시마', '두부', '스낵',\n       '빙과'],\n      dtype='object', length=705571)] are in the [columns]"

In [29]:
# 요일별 과일 전체 판매량 확인

In [20]:
import scipy
import scipy.stats as stats

In [21]:
stats.chi2_contingency(observed=df_fs_category_pivot, correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=102, expected_freq=array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan]]))

### 분석 결과 (기준 : p-value 0.05)
- 통계 측면 결론 :
- 사실 결론 :